In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
!pip install openpyxl
import openpyxl

In [ ]:
df = pd.read_json('data.json')
#print(df.info())
df

In [ ]:
df.to_excel("dataframe.xlsx", index=False)

Index: 4837 entries, 10059 to 10919
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   calibration_measurement_id   4837 non-null   int64  
 1   blind                        4837 non-null   bool   
 2   s3_link                      0 non-null      float64
 3   cancer_tissue                4837 non-null   bool   
 4   tissue_type                  4837 non-null   object 
 5   tissue_size                  0 non-null      float64
 6   exposure_time                4837 non-null   int64  
 7   calibration_date             4837 non-null   object 
 8   calculated_distance          0 non-null      float64
 9   calibration_manual_distance  4837 non-null   int64  
 10  measurement_date             4837 non-null   object 
 11  measurement_manual_distance  0 non-null      float64
 12  code_name                    4837 non-null   object 
 13  cancer_diagnosis             4837 non-null   bool   
 14  hours_since_inoculation      0 non-null      float64
 15  diagnosis                    0 non-null      float64
 16  diag_full                    0 non-null      float64
 17  receptor                     0 non-null      float64
 18  sample_id                    4837 non-null   int64  
 19  sample_type                  0 non-null      float64
 20  study_name                   4837 non-null   object 
 21  target                       4837 non-null   object 
 22  wavelength                   4837 non-null   float64
 23  name                         4837 non-null   object 
 24  machine_id                   4837 non-null   int64  
 25  study_id                     4837 non-null   int64  
 26  patient_id                   4837 non-null   int64  
 27  measurement_orig_file_name   4837 non-null   object 
 28  pixel_size                   4837 non-null   int64  
 29  center_col                   0 non-null      float64
 30  center_row                   0 non-null      float64
 31  helium                       4837 non-null   bool   
 32  background_id                0 non-null      float64
 33  background_orig_file_name    0 non-null      float64
 34  background_date              0 non-null      float64
 35  is_excluded                  4837 non-null   bool   
 36  ponifile                     4837 non-null   object 
 37  measurement_data             4837 non-null   object 

In [ ]:
for index,row in df.iterrows():
    sns.heatmap(row['measurement_data'],robust=True)
    plt.show()

In [ ]:
from collections import Counter
output = []
for index,row in df.iterrows():
    output += [row['calibration_measurement_id']]
counts = Counter(output)
print(counts)

^ separate by calibrations

In [ ]:
from collections import Counter
output = []
for index,row in df.iterrows():
    output += [index]
output.sort()
print(output)

In [ ]:
for index,row in df.iterrows():
    if row['calibration_measurement_id']==729:
        xray_image = np.array(row['measurement_data'])
        plt.imshow(np.log(xray_image+1), cmap='hot')
        plt.title(index)
        plt.show()

        threshold_value = 1000
        primary_beam_mask = xray_image > threshold_value
        true_indices = np.argwhere(primary_beam_mask)
        min_row = true_indices[:, 0].min()
        max_row = true_indices[:, 0].max()
        min_col = true_indices[:, 1].min()
        max_col = true_indices[:, 1].max()
        isolated_beam = np.zeros_like(xray_image)
        isolated_beam[min_row:max_row+1, min_col:max_col+1] = xray_image[min_row:max_row+1, min_col:max_col+1]

        plt.imshow(np.log(isolated_beam+1), cmap='hot')
        plt.title(str(index) + 'primary')
        plt.show()

^^ isolate beam 

In [ ]:
output = {738: [0,0], 737: [0,0], 732: [0,0], 727: [0,0], 728: [0,0], 739: [0,0], 
              731: [0,0], 734: [0,0], 724: [0,0], 747: [0,0], 720: [0,0], 736: [0,0], 
              730: [0,0], 719: [0,0], 748: [0,0], 726: [0,0], 713: [0,0], 722: [0,0], 
              746: [0,0], 745: [0,0], 741: [0,0], 725: [0,0], 743: [0,0], 742: [0,0], 
              716: [0,0], 723: [0,0], 715: [0,0], 729: [0,0], 735: [0,0], 718: [0,0], 
              744: [0,0], 721: [0,0], 717: [0,0], 740: [0,0], 733: [0,0]}
for id in output:
    for index,row in df.iterrows():
        if row['calibration_measurement_id']==id:
            if row['cancer_tissue']:
                output[id][0] += 1
            else:
                output[id][1] += 1
print(output)

^^ t/f for all calibrations

In [ ]:
for index,row in df.iterrows():
    if row['cancer_tissue'] and not row['cancer_diagnosis']:
        print(row['calibration_measurement_id'])